In [1]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta

import time


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")

year_months =[ [2016, 1], [2016, 2], [2016, 3], [2016, 4], [2016, 5], [2016, 6], [2016, 7], [2016, 8], [2016, 9], [2016, 10], [2016, 11], [2016, 12],
    [2017, 1], [2017, 2], [2017, 3], [2017, 4], [2017, 5], [2017, 6], [2017, 7], [2017, 8], [2017, 9], [2017, 10], [2017, 11], [2017, 12],
    [2018, 1], [2018, 2], [2018, 3], [2018, 4], [2018, 5], [2018, 6], [2018, 7], [2018, 8], [2018, 9], [2018, 10], [2018, 11], [2018, 12],[2019, 1], [2019, 2], [2019, 3], [2019, 4], [2019, 5], [2019, 6], [2019, 7]]

# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    adjust = datetime.timedelta(hours=-4)
    
    data = []
    for res in JSONRes['candles']:
        temp = [datetime.datetime.fromisoformat(res['time'][:19]) + adjust]
        
        for i in range(1,13):
           
            if(datetime.datetime.fromisoformat(res['time'][:19]).month==i):
                temp.extend([1]) #月をonehotで追加
                #time.sleep(0.1)
            else:
                temp.extend([0])
                
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).day])  #日を追加
        
        for i in range(0,7):
            
            if(datetime.datetime.fromisoformat(res['time'][:19]).weekday==i):
                temp.extend([1]) #曜日をonehotで追加
                #time.sleep(0.00001)
            else:
                temp.extend([0])
        
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

all_data = []
# year, monthでループ
NY = datetime.timedelta(hours=-13) +  datetime.timedelta(minutes=-1)
for year, month in year_months:
    date_from = datetime.datetime(year, month, 1) +NY
    date_to = date_from + relativedelta(months=+1, day=1) +NY

    ret = getCandleDataFromOanda("USD_JPY", api, date_from, date_to, "M10")
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

#print(all_data)

#ここからall_dataに追加すればよい


# pandas DataFrameへ変換
df = pd.DataFrame(all_data)
df.columns = ['Datetime',
              "January","February","March","April","May","June","July","August","September","October","November","December",
              "day",
              "Sunday","Monday","Thuesday","Wednesday","Thursday","Friday","Saturday",
              'Volume', 'Open', 'High', 'Low', 'Close']
df = df.set_index('Datetime')

#print(df)

#print(df.isnull().any())

print(df)

#cvs への保存
df.to_csv('./USD_JPY_201601-201908_M10.csv')

                     January  February  March  April  May  June  July  August  \
Datetime                                                                        
2015-12-31 11:50:00        0         0      0      0    0     0     0       0   
2015-12-31 12:00:00        0         0      0      0    0     0     0       0   
2015-12-31 12:10:00        0         0      0      0    0     0     0       0   
2015-12-31 12:20:00        0         0      0      0    0     0     0       0   
2015-12-31 12:30:00        0         0      0      0    0     0     0       0   
...                      ...       ...    ...    ...  ...   ...   ...     ...   
2019-06-30 21:10:00        0         0      0      0    0     0     1       0   
2019-06-30 21:20:00        0         0      0      0    0     0     1       0   
2019-06-30 21:30:00        0         0      0      0    0     0     1       0   
2019-06-30 21:40:00        0         0      0      0    0     0     1       0   
2019-06-30 21:50:00        0

In [14]:

year_from = 2010
month_from = 1
year_to = 2019
month_to = 12

year_months = []

for i in range(year_from,year_to+1):
    for j in range(month_from,month_to+1):
        temp = [i, j] 
        year_months.append(temp)

print(year_months)


[[2010, 1], [2010, 2], [2010, 3], [2010, 4], [2010, 5], [2010, 6], [2010, 7], [2010, 8], [2010, 9], [2010, 10], [2010, 11], [2010, 12], [2011, 1], [2011, 2], [2011, 3], [2011, 4], [2011, 5], [2011, 6], [2011, 7], [2011, 8], [2011, 9], [2011, 10], [2011, 11], [2011, 12], [2012, 1], [2012, 2], [2012, 3], [2012, 4], [2012, 5], [2012, 6], [2012, 7], [2012, 8], [2012, 9], [2012, 10], [2012, 11], [2012, 12], [2013, 1], [2013, 2], [2013, 3], [2013, 4], [2013, 5], [2013, 6], [2013, 7], [2013, 8], [2013, 9], [2013, 10], [2013, 11], [2013, 12], [2014, 1], [2014, 2], [2014, 3], [2014, 4], [2014, 5], [2014, 6], [2014, 7], [2014, 8], [2014, 9], [2014, 10], [2014, 11], [2014, 12], [2015, 1], [2015, 2], [2015, 3], [2015, 4], [2015, 5], [2015, 6], [2015, 7], [2015, 8], [2015, 9], [2015, 10], [2015, 11], [2015, 12], [2016, 1], [2016, 2], [2016, 3], [2016, 4], [2016, 5], [2016, 6], [2016, 7], [2016, 8], [2016, 9], [2016, 10], [2016, 11], [2016, 12], [2017, 1], [2017, 2], [2017, 3], [2017, 4], [2017, 5],

In [34]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta

import time


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")

year_from = 2016
month_from = 1
year_to = 2019
month_to = 12

filename = str(year_from) + "_" + str(month_from) + "to" +  str(year_to) + "_" + str(month_to) + ".csv"

year_months = []

for i in range(year_from,year_to+1):
    for j in range(month_from,month_to+1):
        temp = [i, j] 
        year_months.append(temp)
        
        
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    #adjust = datetime.timedelta(hours=-4)
    #adjust = datetime.timedelta(hours=-13)
    
    data = []
    for res in JSONRes['candles']:
        temp = [datetime.datetime.fromisoformat(res['time'][:19])]# + adjust] #indexとなる時間を追加
        
        for i in range(1,13):
           
            if(datetime.datetime.fromisoformat(res['time'][:19]).month==i):
                temp.extend([1]) #月をonehotで追加
                #time.sleep(0.1)
            else:
                temp.extend([0])
                
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).day])  #日を追加
        
        for i in range(0,7):
            
            if(datetime.datetime.fromisoformat(res['time'][:19]).weekday==i):
                temp.extend([1]) #曜日をonehotで追加
                #time.sleep(0.00001)
            else:
                temp.extend([0])
        
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

all_data = []
# year, monthでループ
NY = datetime.timedelta(hours=-13) +  datetime.timedelta(minutes=-1)
for year, month in year_months:
    date_from = datetime.datetime(year, month, 1) +NY
    date_to = date_from + relativedelta(months=+1, day=1) +NY

    ret = getCandleDataFromOanda("USD_JPY", api, date_from, date_to, "M10")
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

#print(all_data)
#print(all_data)

#ここからall_dataに追加すればよい


# pandas DataFrameへ変換
df = pd.DataFrame(all_data)
df.columns = ['Datetime',
              "January","February","March","April","May","June","July","August","September","October","November","December",
              "day",
              "Sunday","Monday","Thuesday","Wednesday","Thursday","Friday","Saturday",
              'Volume', 'Open', 'High', 'Low', 'Close']
df = df.set_index('Datetime')

#print(df)

#print(df.isnull().any())
#print(year_months)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

print(df)

#cvs への保存
df.to_csv(filename)

                     January  February  March  April  May  June  July  August  \
Datetime                                                                        
2016-01-03 22:00:00        1         0      0      0    0     0     0       0   
2016-01-03 22:10:00        1         0      0      0    0     0     0       0   
2016-01-03 22:20:00        1         0      0      0    0     0     0       0   
2016-01-03 22:30:00        1         0      0      0    0     0     0       0   
2016-01-03 22:40:00        1         0      0      0    0     0     0       0   
...                      ...       ...    ...    ...  ...   ...   ...     ...   
2019-12-31 21:10:00        0         0      0      0    0     0     0       0   
2019-12-31 21:20:00        0         0      0      0    0     0     0       0   
2019-12-31 21:30:00        0         0      0      0    0     0     0       0   
2019-12-31 21:40:00        0         0      0      0    0     0     0       0   
2019-12-31 21:50:00        0

In [1]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta

import time


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")

year_from = 2010
month_from = 1
year_to = 2019
month_to = 12

filename = str(year_from) + "_" + str(month_from) + "to" +  str(year_to) + "_" + str(month_to) + ".csv"

year_months = []

for i in range(year_from,year_to+1):
    for j in range(month_from,month_to+1):
        temp = [i, j] 
        year_months.append(temp)
        
        
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    #adjust = datetime.timedelta(hours=-4)
    #adjust = datetime.timedelta(hours=-13)
    
    data = []
    for res in JSONRes['candles']:
        temp = [datetime.datetime.fromisoformat(res['time'][:19])]# + adjust] #indexとなる時間を追加
        
        for i in range(1,13):
           
            if(datetime.datetime.fromisoformat(res['time'][:19]).month==i):
                temp.extend([1]) #月をonehotで追加
                #time.sleep(0.1)
            else:
                temp.extend([0])
                
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).day])  #日を追加
        
        for i in range(0,7):
            
            if(datetime.datetime.fromisoformat(res['time'][:19]).weekday==i):
                temp.extend([1]) #曜日をonehotで追加
                #time.sleep(0.00001)
            else:
                temp.extend([0])
        
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

all_data = []
# year, monthでループ
#NY = datetime.timedelta(hours=-13) +  datetime.timedelta(minutes=-1)
for year, month in year_months:
    date_from = datetime.datetime(year, month, 1)# +NY
    date_to = date_from + relativedelta(months=+1, day=1)# +NY

    ret = getCandleDataFromOanda("USD_JPY", api, date_from, date_to, "M10")
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

#print(all_data)
#print(all_data)

#ここからall_dataに追加すればよい


# pandas DataFrameへ変換
df = pd.DataFrame(all_data)
df.columns = ['Datetime',
              "January","February","March","April","May","June","July","August","September","October","November","December",
              "day",
              "Sunday","Monday","Thuesday","Wednesday","Thursday","Friday","Saturday",
              'Volume', 'Open', 'High', 'Low', 'Close']
df = df.set_index('Datetime')

#print(df)

#print(df.isnull().any())
#print(year_months)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

print(df.isnull().any())

#cvs への保存
df.to_csv(filename)

January      False
February     False
March        False
April        False
May          False
June         False
July         False
August       False
September    False
October      False
November     False
December     False
day          False
Sunday       False
Monday       False
Thuesday     False
Wednesday    False
Thursday     False
Friday       False
Saturday     False
Volume       False
Open         False
High         False
Low          False
Close        False
dtype: bool


In [22]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")

year_from = 2010
month_from = 1
year_to = 2019
month_to = 12

filename = str(year_from) + "_" + str(month_from) + "to" +  str(year_to) + "_" + str(month_to) + ".csv"

year_months = []

for i in range(year_from,year_to+1):
    for j in range(month_from,month_to+1):
        temp = [i, j] 
        year_months.append(temp)
        
        
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    #adjust = datetime.timedelta(hours=-4)
    #adjust = datetime.timedelta(hours=-13)
    
    data = []
    for res in JSONRes['candles']:
        temp = [datetime.datetime.fromisoformat(res['time'][:19])]# + adjust] #indexとなる時間を追加
        
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).hour]) #時間を追加
        
        month = datetime.datetime.fromisoformat(res['time'][:19]).month
        
        for i in range(1,13):
           
            if(month==i):
                temp.extend([1]) #月をonehotで追加
                #time.sleep(0.1)
            else:
                temp.extend([0])
                
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).day])  #日を追加
        
        weekday = datetime.datetime.fromisoformat(res['time'][:19]).weekday
        
        for i in range(0,7):
            
            if(weekday==i):
                temp.extend([1]) #曜日をonehotで追加
                #time.sleep(0.00001)
            else:
                temp.extend([0])
        
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

all_data = []
# year, monthでループ
#NY = datetime.timedelta(hours=-13) +  datetime.timedelta(minutes=-1)
for year, month in year_months:
    date_from = datetime.datetime(year, month, 1)# +NY
    date_to = date_from + relativedelta(months=+1, day=1)# +NY

    ret = getCandleDataFromOanda("USD_JPY", api, date_from, date_to, "M10")
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加


print(np.array(all_data).shape)
print(all_data[0])

#ここからall_dataに追加すればよい


# pandas DataFrameへ変換
df = pd.DataFrame(all_data)

print(df)

df.columns = ['Datetime',"hour_range",
              "January","February","March","April","May","June","July","August","September","October","November","December",
              "day",
              "Sunday","Monday","Thuesday","Wednesday","Thursday","Friday","Saturday",
              'Volume', 'Open', 'High', 'Low', 'Close']
df = df.set_index('Datetime')

#print(df)

#print(df.isnull().any())
#print(year_months)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

print(df.isnull().any())

#cvs への保存
df.to_csv(filename)

(376271, 27)
[datetime.datetime(2010, 1, 3, 18, 0), 18, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 2, '92.943', '93.000', '92.943', '93.000']
                        0   1   2   3   4   5   6   7   8   9   ...  17  18  \
0      2010-01-03 18:00:00  18   1   0   0   0   0   0   0   0  ...   0   0   
1      2010-01-03 18:10:00  18   1   0   0   0   0   0   0   0  ...   0   0   
2      2010-01-03 18:20:00  18   1   0   0   0   0   0   0   0  ...   0   0   
3      2010-01-03 18:30:00  18   1   0   0   0   0   0   0   0  ...   0   0   
4      2010-01-03 18:40:00  18   1   0   0   0   0   0   0   0  ...   0   0   
...                    ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..   
376266 2019-12-31 21:10:00  21   0   0   0   0   0   0   0   0  ...   0   0   
376267 2019-12-31 21:20:00  21   0   0   0   0   0   0   0   0  ...   0   0   
376268 2019-12-31 21:30:00  21   0   0   0   0   0   0   0   0  ...   0   0   
376269 2019-12-31 21:40:00  21   0   0   0   0   0 